In [1]:
import pickle
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import copy
import glob
import transformers
import torch

In [2]:
from transformers import AutoModel, AutoTokenizer, BertTokenizer

In [3]:
tokenizer_filename = 'cambridgeltl/SapBERT-from-PubMedBERT-fulltext'
pt_model = "/data/Bodenreider_UMLS_DL/thilini/EXPERIMENTS/6_SAPBERT_SP/train/out_from_sapbert_from_3/checkpoint-1392096/"

In [4]:
try:
    tokenizer = BertTokenizer(tokenizer_filename)
except:
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_filename)
    
model = AutoModel.from_pretrained(pt_model)

Some weights of the model checkpoint at /data/Bodenreider_UMLS_DL/thilini/EXPERIMENTS/6_SAPBERT_SP/train/out_from_sapbert_from_3/checkpoint-1392096/ were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
query = 'Membership'
false_negative = 'memberships'
false_positive = 'gallbladder disease 4'
false_positive2 = 'gallbladder'
false_positive3 = 'disease 4'

In [42]:
query = 'Bullying, Cyber'
false_negative = 'Cyberbullying'
false_positive = 'trimebutine'
false_positive2 = 'Reason for assessment (observable entity)'
false_positive3 = 'Reason for assessment'

In [12]:
text_batch = [query, false_negative, false_positive, false_positive2, false_positive3]

In [13]:
with torch.no_grad():
    encoding = tokenizer(text_batch, return_tensors='pt', padding=True, truncation=True,max_length=model.config.max_length)
    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']

    outputs = model(input_ids, attention_mask=attention_mask)

In [14]:
o = outputs[0][:,0,:].numpy()

In [15]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import manhattan_distances

In [16]:
for i,word in enumerate(text_batch):
    print(text_batch[0], word)
    print(cosine_similarity(o[[0]],o[[i]])[0,0])
    print(manhattan_distances(o[[0]],o[[i]])[0,0])
    print()

Membership Membership
1.0
0.0

Membership memberships
0.9746059
46.52413746831007

Membership gallbladder disease 4
0.99748904
14.34048222318961

Membership gallbladder
0.6948005
193.81846785095695

Membership disease 4
0.43971342
273.6131640996464



In [71]:
cosine_similarity(o[[0]],o[[1]])

array([[0.8871909]], dtype=float32)

In [72]:
cosine_similarity(o[[0]],o[[2]])

array([[0.89589125]], dtype=float32)

In [73]:
cosine_similarity(o[[0]],o[[3]])

array([[0.9454955]], dtype=float32)